# Load initial data to SQL

In this file, we perform the proccess of reading and loading from a paquet file obtained by api of Hugging Face about all games available on the Steam platform using Python Kernel in a venv.

In this process, the following will be carried out:

1. Connection with the **steam_database** database hosted in a **Local server**. If the database doesn't exists, it will be created.

2. Creation of the database's structure if the database doesn't exists. As the proccess is running, the tables will be reset if they already exists.

3. Reading and exploration of paquet file of Hugging Face to knows its structure and data.

4. Loading of data to **raw_games** table in **steam_database** database.

---
# Proccess

## 1. Environment Setup

### Installing missing libraries

In [ ]:
pip install -r ../code/config/requirements.txt

### Import modules helper:

In [8]:
import importlib.util

def import_module_from_path(module_name, path):
    spec = importlib.util.spec_from_file_location(module_name, path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

### Import Utils & DB Connection

In [26]:
connect_database = import_module_from_path("connect_database", "../code/utils/connect_database.py")

### Loading libraries

In [10]:
import pandas as pd

## 2. Data Preparation

### Let's know the dataset

Before reading the data, we downloaded the repository and it presented us with a README that brings us the column metadata.

Information about parquet file to load:

In [11]:
location_file = 'https://huggingface.co/api/datasets/FronkonGames/steam-games-dataset/parquet/default/train/0.parquet'
raw_table_database = 'raw_games'
columns_to_delete = []

Check the data to know the structure and lengths of the data for the **raw_games** table that will store the raw data.

In [12]:
raw_games_dataframe = pd.read_parquet(location_file)

Let's see the columns to verify its names and if it's necessary, rename them to load in the **steam_database** database.

In [13]:
raw_games_dataframe.shape

(83560, 39)

In [14]:
raw_games_dataframe.columns

Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

Let's rename columns to prepare the load for database.

In [15]:
names_file = ['AppID', 'name', 'release_date', 'estimated_owners', 'peak_ccu', 'required_age', 'price', 'downloable_content_count', 'about_the_game','supported_languages', 'full_audio_languages', 'reviews', 'header_image', 'website', 'support_url', 'support_email', 'windows', 'mac', 'linux', 'metacritic_score', 'metacritic_url', 'user_score', 'positive', 'negative', 'score_rank', 'achievements', 'recommendations', 'notes', 'average_playtime_forever_minute', 'average_playtime_two_weeks_minute', 'median_playtime_forever_minute', 'median_playtime_two_weeks_minute', 'developers', 'publishers', 'categories', 'genres', 'tags', 'screenshots', 'movies']
raw_games_dataframe.columns = names_file

AppID is the unique identifier for each. We will see the range of identifiers to use them as **id** in the table or we need to create them:

In [16]:
len(raw_games_dataframe['AppID'].unique())

83560

In [17]:
"Min: %s - Max: %s" % (str(raw_games_dataframe['AppID'].min()), str(raw_games_dataframe['AppID'].max()))

'Min: 10 - Max: 2719700'

For 83.560 rows, an identifier number 2.719.700 is very great, so we will use the indexes provided by **Pandas** to assign the identifiers and the **AppID** column will be marked for later deletion.

In [18]:
raw_games_dataframe.index += 1
raw_games_dataframe.reset_index(inplace=True)
raw_games_dataframe.rename(columns={'index': 'id'}, inplace=True)
columns_to_delete.append('AppID')
raw_games_dataframe.columns

Index(['id', 'AppID', 'name', 'release_date', 'estimated_owners', 'peak_ccu',
       'required_age', 'price', 'downloable_content_count', 'about_the_game',
       'supported_languages', 'full_audio_languages', 'reviews',
       'header_image', 'website', 'support_url', 'support_email', 'windows',
       'mac', 'linux', 'metacritic_score', 'metacritic_url', 'user_score',
       'positive', 'negative', 'score_rank', 'achievements', 'recommendations',
       'notes', 'average_playtime_forever_minute',
       'average_playtime_two_weeks_minute', 'median_playtime_forever_minute',
       'median_playtime_two_weeks_minute', 'developers', 'publishers',
       'categories', 'genres', 'tags', 'screenshots', 'movies'],
      dtype='object')

Let's prepare the lengths for each column if it's char

In [19]:
max_lengths = raw_games_dataframe.apply(lambda col: "{} - {}" .format(col.dtypes, col.astype(str).str.len().max()))

In [20]:
max_lengths

id                                        int64 - 5
AppID                                     int64 - 7
name                                   object - 184
release_date                            object - 12
estimated_owners                        object - 21
peak_ccu                                  int64 - 6
required_age                              int64 - 2
price                                   float64 - 6
downloable_content_count                  int64 - 4
about_the_game                       object - 94466
supported_languages                   object - 1216
full_audio_languages                  object - 1216
reviews                               object - 2911
header_image                            object - 90
website                                object - 206
support_url                            object - 349
support_email                          object - 247
windows                                    bool - 5
mac                                        bool - 5
linux       

In [21]:
pd.set_option('display.max_columns', None)
raw_games_dataframe.head()

,id,AppID,name,release_date,estimated_owners,peak_ccu,required_age,price,downloable_content_count,about_the_game,supported_languages,full_audio_languages,reviews,header_image,website,support_url,support_email,windows,mac,linux,metacritic_score,metacritic_url,user_score,positive,negative,score_rank,achievements,recommendations,notes,average_playtime_forever_minute,average_playtime_two_weeks_minute,median_playtime_forever_minute,median_playtime_two_weeks_minute,developers,publishers,categories,genres,tags,screenshots,movies
0,1,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],[],None,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.galacticbowling.net,None,None,True,False,False,0,None,0,6,11,NaN,30,0,None,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,2,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",[],None,https://cdn.akamai.steamstatic.com/steam/apps/...,http://trainbandit.com,None,support@rustymoyher.com,True,True,False,0,None,0,53,5,NaN,12,0,None,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,3,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",[],None,https://cdn.akamai.steamstatic.com/steam/apps/...,None,None,ramoncampiaof31@gmail.com,True,False,False,0,None,0,0,0,NaN,0,0,None,0,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",None,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,4,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",[],None,https://cdn.akamai.steamstatic.com/steam/apps/...,https://henosisgame.com/,https://henosisgame.com/,info@henosisgame.com,True,True,True,0,None,0,3,0,NaN,0,0,None,0,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,5,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",[],None,https://cdn.akamai.steamstatic.com/steam/apps/...,https://www.unusual-games.com/home/,https://www.unusual-games.com/contact/,welistentoyou@unusual-games.com,True,True,False,0,None,0,50,8,NaN,17,0,This Game may contain content not appropriate ...,0,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In order to perform the estimation of the prediction model and with this information, we need to choose which columns to take to create the database table model:

Columns to load:

- **id**
- **name**
- **release_date**
- **estimated_owners**
- **peak_ccu**
- **required_age**
- **price**
- **downloable_content_count**
- **supported_languages**
- **full_audio_languages**
- **reviews**
- **website**
- **support_url**
- **support_email**
- **windows**
- **mac**
- **linux**
- **metacritic_score**
- **metacritic_url**
- **user_score**
- **positive**
- **negative**
- **score_rank**
- **achievements**
- **recomendations**
- **average_playtime_forever**
- **average_playtime_two_weeks**
- **median_playtime_forever_weeks**
- **median_playtime_two_ weeks**
- **developers**
- **publishers**
- **categories**
- **genres**
- **tags**

Columns not to load:

- **about_the_game**: Its records have information about the game in written form, so it is not in our interest to draw metrics for all descriptions.
- **header_image**: Its records have header image urls, so it is not information of interest to the scope.
- **Notes**: Its records have extra information about the game content in written form, so it is not within our scope. 
- **screenshots**: Its records have game screenshot urls; as they are image formats, they will not be treated within our scope.
- **movies**: Its records have game movie urls; therefore, it will not be treated within our scope.

We will mark the columns that will be deleted because they will not be loaded to the **raw_games** table:

In [22]:
columns_to_delete += ['about_the_game', 'header_image', 'notes', 'screenshots', 'movies']

Let's create the **raw_games** model table.

The model structure is:

- **id** <- Integer, PK
- **name** <- Varchar (200), default (None)
- **release_date** <- Varchar (12), default (None)
- **estimated_owners** <- Varchar (30), default (None)
- **peak_ccu** <- Varchar (20), default (None)
- **required_age** <- Varchar (3), default (None)
- **price** <- Varchar (20), default (None)
- **downloable_content_count** <- Varchar (20), default (None)
- **supported_languages** <- Varchar (1500), default (None)
- **full_audio_languages** <- Varchar (1500), default (None)
- **reviews** <- Varchar (4000), default (None)
- **website** <- Varchar (500), default (None)
- **support_url** <- Varchar (500), default (None)
- **support_email** <- Varchar (500), default (None)
- **windows** <- Varchar (20), default (None)
- **mac** <- Varchar (20), default (None)
- **linux** <- Varchar (20), default (None)
- **metacritic_score** <- Varchar (20), default (None)
- **metacritic_url** <- Varchar (20), default (None)
- **user_score** <- Varchar (20), default (None)
- **positive** <- Varchar (20), default (None)
- **negative** <- Varchar (20), default (None)
- **score_rank** <- Varchar (20), default (None)
- **achievements** <- Varchar (20), default (None)
- **recommendations** <- Varchar (20), default (None)
- **average_playtime_forever_minute** <- Varchar (20), default (None)
- **average_playtime_two_weeks_minute** <- Varchar (20), default (None)
- **median_playtime_forever_minute** <- Varchar (20), default (None)
- **median_playtime_two_weeks_minute** <- Varchar (20), default (None)
- **developers** <- Varchar (800), default (None)
- **publishers** <- Varchar (800), default (None)
- **categories** <- Varchar (500), default (None)
- **genres** <- Varchar (500), default (None)
- **tags** <- Varchar (500), default (None)

Let's delete the columns marked:

In [23]:
raw_games_dataframe.drop(columns_to_delete, axis=1, inplace=True)

### Let's load the data to PostgreSQL

Create connection:

In [24]:
connection = connect_database.ConnectionPostgres()
connection.make_tables()

Load data to SQL:

In [25]:
raw_games_dataframe.to_sql(raw_table_database, connection.engine, if_exists='replace', index=False)
connection.log('Data loaded to {}: {} rows - {} columns.' .format(connection.connection_config['database'], raw_games_dataframe.shape[0], raw_games_dataframe.shape[1]))

connection.close_connection()

---
### Evidence

- Database created:

    ![Database created](https://gist.github.com/dventep/fc64df11bc7aeb08ce9c231152970221/raw/05ada9adc7ea107697b7ae87389c051550228ba5/database_created.png)

- Records loaded in **raw_games** table:

    ![Records loaded](https://gist.github.com/dventep/fc64df11bc7aeb08ce9c231152970221/raw/05ada9adc7ea107697b7ae87389c051550228ba5/records_loaded_in_table.png)